##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 케라스

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/r2/guide/keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/r2/guide/keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도
불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다.
이 번역에 개선할 부분이 있다면
[tensorflow/docs](https://github.com/tensorflow/docs) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다.
문서 번역이나 리뷰에 지원하려면 [이 양식](https://bit.ly/tf-translate)을
작성하거나
[docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)로
메일을 보내주시기 바랍니다.

케라스(Keras)는 딥러닝 모델을 만들고 훈련하기 위한 고수준 API입니다. 빠른 프로토타이핑과 최신 연구, 제품 개발에 사용됩니다. 케라스의 세 가지 장점은 다음과 같습니다:

- *사용하기 편합니다.*<br>
  케라스는 일반적인 사용 방식에 최적화된 간단하고 일관된 인터페이스를 제공합니다. 사용자에게 명확하고 확인 가능한 오류를 피드팩합니다.
- *모듈화되어 있어 조합 가능합니다.*<br>
  케라스 모델은 거의 제약없이 설정 변경 가능한 케라스 구성 요소들을 서로 연결하여 만듭니다.
- *확장하기 쉽습니다.*<br>
  새로운 연구 아이디어를 실험해 보기 위해 자신만의 구성 요소를 만들 수 있습니다. 새로운 층, 새로운 손실 함수를 만들어 최상의 모델을 개발할 수 있습니다.

## tf.keras 임포트

`tf.keras`는 [케라스 API 명세](https://keras.io){:.external}의 텐서플로 구현입니다. `tf.keras`는 머신러닝 모델을 만들고 훈련하기 위한 고수준 API로서 텐서플로의 특수 기능을 모두 지원합니다. 여기에는 [즉시 실행](#eager_execution), `tf.data` 파이프라인(pipeline), [Estimators](./estimators.md)가 포함됩니다. `tf.keras`를 이용하면 유연성과 성능을 손해보지 않고 텐서플로를 쉽게 사용할 수 있습니다.

`tf.keras`를 임포트하여 텐서플로 프로그램을 시작합니다:

In [0]:
!pip install -q pyyaml  # pyyaml은 선택사항입니다.

In [0]:
from __future__ import absolute_import, division, print_function

!pip install tf-nightly-2.0-preview
import tensorflow as tf

from tensorflow.keras import layers

print(tf.__version__)
print(tf.keras.__version__)

`tf.keras`는 케라스 API와 호환되는 어떤 코드라도 실행시킬 수 있지만 다음 사항을 유념하세요:

* 최신 텐서플로 릴리스에 포함된 `tf.keras` 버전은 PyPI에 있는 최신 `keras` 버전과 같지 않을 수 있습니다. `tf.keras.__version__`을 확인해 보세요.
* [모델의 가중치를 저장](#weights_only)할 때 `tf.keras`는 기본적으로 [체크포인트 포맷](./checkpoints.md)을 사용합니다. HDF5를 사용하려면 `save_format='h5'`로 설정하세요.

## 간단한 모델 만들기

### Sequential 모델

케라스에서는 *층(layer)*을 조합하여 *모델(model)*을 만듭니다. 모델은 (일반적으로) 층의 그래프입니다. 가장 흔한 모델 구조는 층을 차례대로 쌓은 `tf.keras.Sequential` 모델입니다.

간단한 완전 연결(fully-connected) 네트워크(즉, 다층 퍼셉트론(multi-layer perceptron))를 만들어 보겠습니다.

In [0]:
model = tf.keras.Sequential()
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 또 하나를 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
model.add(layers.Dense(10, activation='softmax'))

### 층 설정

`tf.keras.layers`에는 많은 생성자 매개변수가 있습니다:

* `activation`: 층의 활성화 함수를 설정합니다. 이 매개변수에 기본으로 제공되는 함수의 이름을 쓰거나 
  호출 가능한 객체를 지정할 수 있습니다. 기본값은 활성화 함수를 적용하지 않습니다.
* `kernel_initializer`와 `bias_initializer`: 층의 가중치(weight)(커널(kernel)과 절편(bias))를 초기화하는 방법입니다. 내장 함수나 호출 가능한 객체를 지정합니다. 기본값은 `"Glorot uniform"` 초기화입니다.
* `kernel_regularizer`와 `bias_regularizer`: L1 또는 L2 규제(regularization)와 같이 층의 가중치(커널과 절편)에 적용할 규제 방법을 지정합니다. 기본값은 규제를 적용하지 않습니다.

다음 코드는 여러가지 생성자 매개변수를 사용하여 `tf.keras.layers.Dense` 층의 객체를 만듭니다:

In [0]:
# 시그모이드 층을 만듭니다:
layers.Dense(64, activation='sigmoid')
# 또는 다음도 가능합니다:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 커널 행렬에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 절편 벡터에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 커널을 랜덤한 직교 행렬로 초기화한 선형 활성화 층:
layers.Dense(64, kernel_initializer='orthogonal')

# 절편 벡터를 상수 2.0으로 설정한 선형 활성화 층:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## 훈련과 평가

### 훈련 준비

모델은 구성한 후 `compile` 메서드를 호출하여 학습 과정을 설정합니다:

In [0]:
model = tf.keras.Sequential([
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
layers.Dense(64, activation='relu'),
# 또 하나를 추가합니다:
layers.Dense(64, activation='relu'),
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

`tf.keras.Model.compile`에는 세 개의 중요한 매개변수가 있습니다:

* `optimizer`: 훈련 과정을 설정합니다. `tf.keras.optimizers.Adam`이나
  `tf.keras.optimizers.SGD`와 같은 `tf.keras.optimizers` 아래의 옵티마이저 객체를 전달합니다. 기본 매개변수를 사용한다면 `'adam'`이나 `'sgd'`와 같이 문자열로 지정할 수도 있습니다.
* `loss`: 최적화 과정에서 최소화될 손실 함수. 평균 제곱 오차(`mse`)와 `categorical_crossentropy`, `binary_crossentropy` 등이 자주 사용됩니다. 손실 함수(loss function)는 이름으로 지정하거나 `tf.keras.losses` 모듈 아래의 호출 가능한 객체를 전달할 수 있습니다.
* `metrics`: 훈련을 모니터링하기 위해 사용됩니다. 이름이나 `tf.keras.metrics` 모듈 아래의 호출 가능한 객체입니다.
* 추가적으로 모델의 훈련과 평가를 즉시 실행하려면 `run_eagerly=True` 매개변수를 전달할 수 있습니다.

다음 코드는 모델 훈련을 설정하는 몇 가지 예를 보여줍니다:

In [0]:
# 평균 제곱 오차로 회귀 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # 평균 제곱 오차
              metrics=['mae'])  # 평균 절댓값 오차

# 크로스엔트로피 손실 함수로 분류 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

### 넘파이 데이터 입력

작은 데이터셋일 경우 [넘파이](https://www.numpy.org/){:.external}(NumPy) 배열을 메모리에 적재하여 모델을 훈련하고 평가합니다. 모델은 `fit` 메서드를 사용하여 훈련 데이터를 학습합니다:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

`tf.keras.Model.fit`에는 세 개의 중요한 매개변수가 있습니다:

* `epochs`: 훈련은 *에포크*(epoch)로 구성됩니다. 한 에포크는 전체 입력 데이터를 한번 순회하는 것입니다(작은 배치로 나누어 수행됩니다).
* `batch_size`: 넘파이 데이터를 전달하면 모델은 데이터를 작은 배치로 나누고 훈련 과정에서 이 배치를 순회합니다. 이 정수 값은 배치의 크기를 지정합니다. 전체 샘플 개수가 배치 크기로 나누어 떨어지지 않으면 마지막 배치의 크기는 더 작을 수 있습니다.
* `validation_data`: 모델의 프로토타입(prototype)을 만들 때 검증 데이터(validation data)에서 성능을 쉽게 모니터링하는 것이 좋습니다. 입력과 레이블(label)의 튜플을 이 매개변수로 전달하면 에포크가 끝날 때마다 모델이 추론 모드에서 전달된 데이터의 손실과 측정 지표를 출력합니다.

다음이 `validation_data`를 사용하는 예입니다:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

### tf.data 데이터셋 입력

[데이터셋 API](./datasets.md)를 사용하여 대규모 데이터셋이나 복수의 장치로 확장시킬 수 있습니다. `fit` 메서드에 `tf.data.Dataset` 객체를 전달합니다:

In [0]:
# 예제 `Dataset` 객체를 만듭니다:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

# Dataset에서 `fit` 메서드를 호출할 때 `steps_per_epoch` 설정을 잊지 마세요.
model.fit(dataset, epochs=10, steps_per_epoch=30)

여기에서 `fit` 메서드는 `steps_per_epoch` 매개변수를 사용합니다. 다음 에포크로 넘어가기 전에 모델이 수행할 훈련 단계 횟수입니다. `Dataset`은 데이터의 배치를 생성하기 때문에 `batch_size`가 필요하지 않습니다.

`Dataset`은 검증 데이터에도 사용할 수 있습니다:

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

### 평가와 예측

`tf.keras.Model.evaluate`와 `tf.keras.Model.predict` 메서드에는 넘파이 배열이나 `tf.data.Dataset`을 사용할 수 있습니다.

데이터를 주입하여 추론 모드에서의 손실이나 지표를 *평가*합니다:

In [0]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

model.evaluate(dataset, steps=30)

데이터를 주입하여 추론 모드에서 마지막 층의 출력에 대한 *예측*을 넘파이 배열로 출력합니다:

In [0]:
result = model.predict(data, batch_size=32)
print(result.shape)

## 고급 모델 만들기

### 함수형 API

`tf.keras.Sequential` 모델은 단순히 층을 쌓은 것으로 임의의 구조를 표현할 수 없습니다. [케라스 함수형 API](https://keras.io/getting-started/functional-api-guide/){:.external}을 사용하면 다음과 같은 복잡한 모델 구조를 만들 수 있습니다:

* 다중 입력 모델,
* 다중 출력 모델,
* 층을 공유하는 모델 (동일한 모델을 여러번 호출합니다),
* 데이터 흐름이 차례대로 진행되지 않는 모델 (예를 들면 잔차 연결(residual connections)).

함수형 API로 모델을 만드는 과정은 다음과 같습니다:

1. 층의 객체는 호출 가능하고 텐서를 반환합니다.
2. `tf.keras.Model` 객체를 정의하기 위해 입력 텐서와 출력 텐서를 사용합니다.
3. 이 모델은 `Sequential` 모델과 동일한 방식으로 훈련됩니다.

다음 코드는 함수형 API를 사용하여 간단한 완전 연결 네트워크를 만드는 예입니다:

In [0]:
inputs = tf.keras.Input(shape=(32,))  # 입력 플레이스홀더를 반환합니다.

# 층 객체는 텐서를 사용하여 호출되고 텐서를 반환합니다.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

입력과 출력을 사용해 모델의 객체를 만듭니다.

In [0]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5번의 에포크 동안 훈련합니다.
model.fit(data, labels, batch_size=32, epochs=5)

### 모델 클래스 상속

`tf.keras.Model` 클래스를 상속하고 자신만의 정방향 계산(forward pass)을 정의하여 완전히 커스터마이징된 모델을 만들 수 있습니다. `__init__` 메서드에서 층을 만들어 클래스 객체의 속성으로 지정하세요. 정방향 계산은 `call` 메서드에 정의합니다.

정방향 계산을 순서대로 쓸 수 있기 때문에 [즉시 실행](./eager.md)이 활성화되어 있을 때 모델 클래스 상속이 특히 유용합니다.

노트: 정방향 계산을 *항상* 순서대로 실행하려면 `super` 객체의 생성자를 호출할 때 `dynamic=True`를 지정하세요.

중요 포인트: 작업에 맞는 API를 사용하세요. 모델 클래스 상속은 유연성을 제공하지만 복잡도가 증가하고 사용자 에러가 발생할 가능성이 높아집니다. 가능한 함수형 API를 사용하세요.

다음 코드는 `tf.keras.Model`의 클래스를 상속하여 순서대로 실행할 필요가 없는 정방향 계산을 구현한 예입니다:

In [0]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # 층을 정의합니다.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 정방향 계산을 정의합니다.
    # `__init__` 메서드에서 정의한 층을 사용합니다.
    x = self.dense_1(inputs)
    return self.dense_2(x)

  def compute_output_shape(self, input_shape):
    # 상속한 모델을 함수형 API 스타일로 사용하려면 이 함수를 오버라이드해야합니다.
    # 그렇지 않다면 이 메서드는 선택 사항입니다.
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.num_classes
    return tf.TensorShape(shape)

새 모델 클래스의 객체를 만듭니다:

In [0]:
model = MyModel(num_classes=10)

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5번의 에포크 동안 훈련합니다.
model.fit(data, labels, batch_size=32, epochs=5)

다음 코드는 동적인 입력 크기를 사용하고 순서대로 정방향 계산을 실행해야 하는 RNN 예제입니다.

In [0]:
class MyRNN(tf.keras.Model):

  def __init__(self, units=16):
    # 정방향 계산을 즉시 실행 모드로 실행되게 하려면 dynamic=True로 설정합니다.
    super(MyRNN, self).__init__(dynamic=True)
    self.units = units
    self.projection_1 = layers.Dense(units=units, activation='tanh')
    self.projection_2 = layers.Dense(units=units, activation='tanh')

  def call(self, inputs):
    outputs = []
    state = tf.zeros(shape=(inputs.shape[0], self.units))
    for t in range(inputs.shape[1]):
        x = inputs[:, t, :]
        h = self.projection_1(x)
        y = h + self.projection_2(state)
        state = y
        outputs.append(y)
    return tf.stack(outputs, axis=1)

rnn = MyRNN()

# 컴파일 단계는 훈련 과정을 설정합니다.
rnn.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy')

rnn_data = np.random.random((30, 10, 32))
rnn_labels = np.random.random((30, 10, 16))

# 5번의 에포크 동안 훈련합니다.
rnn.fit(rnn_data, rnn_labels, batch_size=1, epochs=5)

### Custom layers

Create a custom layer by subclassing `tf.keras.layers.Layer` and implementing
the following methods:

* `build`: Create the weights of the layer. Add weights with the `add_weight`
  method.
* `call`: Define the forward pass.
* `compute_output_shape`: Specify how to compute the output shape of the layer
  given the input shape.
* Optionally, a layer can be serialized by implementing the `get_config` method
  and the `from_config` class method.

Here's an example of a custom layer that implements a `matmul` of an input with
a kernel matrix:

In [0]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # Create a trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Be sure to call this at the end
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

Create a model using your custom layer:

In [0]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

## Callbacks

A callback is an object passed to a model to customize and extend its behavior
during training. You can write your own custom callback, or use the built-in
`tf.keras.callbacks` that include:

* `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at
  regular intervals.
* `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning
  rate.
* `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation
  performance has stopped improving.
* `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using
  [TensorBoard](./summaries_and_tensorboard.md).

To use a `tf.keras.callbacks.Callback`, pass it to the model's `fit` method:

In [0]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

## Save and restore

### Weights only

Save and load the weights of a model using `tf.keras.Model.save_weights`:

In [0]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# this requires a model with the same architecture.
model.load_weights('./weights/my_model')

By default, this saves the model's weights in the
[TensorFlow checkpoint](./checkpoints.md) file format. Weights can
also be saved to the Keras HDF5 format (the default for the multi-backend
implementation of Keras):

In [0]:
# Save weights to a HDF5 file
model.save_weights('my_model.h5', save_format='h5')

# Restore the model's state
model.load_weights('my_model.h5')

### Configuration only

A model's configuration can be saved—this serializes the model architecture
without any weights. A saved configuration can recreate and initialize the same
model, even without the code that defined the original model. Keras supports
JSON and YAML serialization formats:

In [0]:
# Serialize a model to JSON format
json_string = model.to_json()
json_string

In [0]:
import json
import pprint
pprint.pprint(json.loads(json_string))

Recreate the model (freshly initialized), from the json.

In [0]:
fresh_model = tf.keras.models.model_from_json(json_string)

Serializing a model to YAML format requires that you install `pyyaml` *before you import TensorFlow*:

In [0]:
yaml_string = model.to_yaml()
print(yaml_string)

Recreate the model from the yaml

In [0]:

fresh_model = tf.keras.models.model_from_yaml(yaml_string)

Caution: Subclassed models are not serializable because their architecture is
defined by the Python code in the body of the `call` method.


### Entire model

The entire model can be saved to a file that contains the weight values, the
model's configuration, and even the optimizer's configuration. This allows you
to checkpoint a model and resume training later—from the exact same
state—without access to the original code.

In [0]:
# Create a trivial model
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)


# Save entire model to a HDF5 file
model.save('my_model.h5')

# Recreate the exact same model, including weights and optimizer.
model = tf.keras.models.load_model('my_model.h5')

## Eager execution

[Eager execution](./eager.md) is an imperative programming
environment that evaluates operations immediately. This is not required for
Keras, but is supported by `tf.keras` and useful for inspecting your program and
debugging.

All of the `tf.keras` model-building APIs are compatible with eager execution.
And while the `Sequential` and functional APIs can be used, eager execution
especially benefits *model subclassing* and building *custom layers*—the APIs
that require you to write the forward pass as code (instead of the APIs that
create models by assembling existing layers).

See the [eager execution guide](./eager.ipynb#build_a_model) for
examples of using Keras models with custom training loops and `tf.GradientTape`.

## Distribution

### Multiple GPUs

`tf.keras` models can run on multiple GPUs using
`tf.distribute.Strategy`. This API provides distributed
training on multiple GPUs with almost no changes to existing code.

Currently, `tf.distribute.MirroredStrategy` is the only supported
distribution strategy. `MirroredStrategy` does in-graph replication with
synchronous training using all-reduce on a single machine. To use
`distribute.Strategy`s with Keras, nest the optimizer instantiation and model construction and compilation in a `Strategy`'s `.scope()`, then
train the model.

The following example distributes a `tf.keras.Model` across multiple GPUs on a
single machine.

First, define a model inside the distributed strategy scope:

In [0]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = tf.keras.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
  model.add(layers.Dense(1, activation='sigmoid'))

  optimizer = tf.keras.optimizers.SGD(0.2)

  model.compile(loss='binary_crossentropy', optimizer=optimizer)

model.summary()

Next, train the model on data like normal:

In [0]:
  x = np.random.random((1024, 10))
  y = np.random.randint(2, size=(1024, 1))
  x = tf.cast(x, tf.float32)
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.repeat(10)
  dataset = dataset.batch(32)

  model.fit(dataset, epochs=1, steps_per_epoch=10)

For more information, see the [full guide on Distributed Training in TensorFlow](distribute_strategy.ipynb).